<a href="https://colab.research.google.com/github/lups0208/datasciencecoursera/blob/master/oferta_hidro_diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=9297c103eb16c374ed9f8383db267c71661808071f908b1c99bbbd14832cec7f
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget
from zipfile import ZipFile
import os
import pandas as pd
import re #Esta libreria nos permite buscar patrones en strings
from datetime import datetime

In [ ]:
#URLs Github 
SHP_ROOT = 'https://github.com/IvanMGro/CENACE/raw/main/'
SHP_FILE = 'ofertas_hidro_diaria.zip'

#Damos como argumento el url y descargamos los zipfiles
wget.download(SHP_ROOT+SHP_FILE)

#Creamos un objeto ZipFile mediante el modulo ZipFile el cual le damos
#como argumento nuestro archivo zip, 'r' es un parámetro para indicar 
#que abriremos el zip en modo de lectura (read mode) y harémos referencia
#a este archivo bajo el alias 'zip' ('as zip') 
with ZipFile(SHP_FILE, 'r') as zip:
    #Extraemos todo el contenido del zip con el metodo .extractall()
    print('Extrayendo archivos...')
    zip.extractall()
    print('Finalizado')

Extrayendo archivos...
Finalizado


Accedemos a la carpeta con los datos

In [ ]:
os.chdir('2021')

Creamos la lista que contiene los archivos a leer de las ofertas hidroeléctrica

In [ ]:
dirs = os.listdir()
sorted_dirs = sorted(dirs) #ordeno la lista
ofertas = pd.DataFrame()
sorted_dirs

Iteramos y concatenamos todos los dataframes leídos en el dataframe ```ofertas```. Buscamos dentro del string ```file```el patron ```dddd-dd-dd```, donde ```d``` representa un digito cualquiera.



Para más infomración -> https://docs.hektorprofe.net/python/funcionalidades-avanzadas/expresiones-regulares/

https://docs.python.org/es/3/library/re.html (Documentación original)

In [ ]:
for file in sorted_dirs:        
    #Se trabaja cada archivo csv para obtenerlo en el formato final deseado
    file_date=re.search('\d{4}-\d{2}-\d{2}', file) #busca aaaa-mm-dd 
    #Busca dentro del string del archivo el patron aaaa-mm-dd 
    date = datetime.strptime(file_date.group(), '%Y-%m-%d').date() 
    df_day = pd.read_csv(file,header=8,index_col=False)
    print(date)
    df_day['Fecha_Operacion'] = date
    df_day.dropna(inplace=True)
    ofertas = pd.concat([ofertas,df_day])

In [ ]:
prom_diario = ofertas.groupby([' Grupo de Unidades (Embalse)','Fecha_Operacion'])[' Costo de Oportunidad ($/MWh)'].mean().reset_index()
prom_diario

,Grupo de Unidades (Embalse),Fecha_Operacion,Costo de Oportunidad ($/MWh)
0,AGUAMILPA,2021-01-01,1256.28
1,AGUAMILPA,2021-01-02,1256.28
2,AGUAMILPA,2021-01-03,1256.28
3,AGUAMILPA,2021-01-04,1256.28
4,AGUAMILPA,2021-01-05,1256.28
...,...,...,...
5147,ZIMAPAN,2021-11-14,451.94
5148,ZIMAPAN,2021-11-15,451.94
5149,ZIMAPAN,2021-11-16,451.94
5150,ZIMAPAN,2021-11-17,451.94


In [ ]:
import plotly.express as px
fig = px.line(prom_diario,
              x='Fecha_Operacion',
              y=' Costo de Oportunidad ($/MWh)',
              color=' Grupo de Unidades (Embalse)',
              title='Costos de Oportunidad 2021 ')
fig.show()